<center><br><br>
    <h4>TANF Data Collaborative </h4>
    <h4>Applied Data Analytics Training | Spring 2022</h4>
    <h1>Creating the Analytical Frame </h1>
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Coleridge Initiative</a>
    </span>
    <center>Maryah Garner, Allison Nunez, Carolyn D. Gorman</center>
    <a href="https://doi.org/10.5281/zenodo.7459860"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.7459860.svg" alt="DOI"></a>
</center>

<br>


In [ ]:
library(odbc)                       # allows R to connect with the database
library(tidyverse)                  # useful for data manipulation and visualization
library(scales)                     # to calculate percentages, graphing
library(lubridate)                  # for easy working with dates 
library(zoo)                        # for creating a quarter variable

# 
Connect to the database

In [ ]:
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

In [ ]:
query <- "
SELECT 
    ssn, caseid, month AS yr_month, tanf_end
FROM ds_in_fssa.dbo.person_month
;
"

df <- dbGetQuery(con, query)   

head(df)

## Remove bad SSNs

Identifying bad SSNs

In [ ]:
df %>%
    group_by(ssn) %>%
    summarise(cases =  n_distinct(caseid)) %>%
    arrange(desc(cases)) %>%
    head(3)
        
    

Identifying bad SSNs casses

Note, those first who SSN's are obviously bad.  

In [ ]:
# calculate the number of casses each person has each month
df <- df %>%
    filter(!ssn %in% c('REDACTED', 'REDACTED'),
          !is.na(yr_month)) %>%
    group_by(ssn, yr_month) %>%
    mutate(cases = n_distinct(caseid)) %>%
    ungroup() 
head(df)

## Remove people who have more than one case in a given month

In [ ]:
temp <- df %>%
    group_by(ssn, yr_month) %>%
    summarise(count = n()) %>%
    ungroup() %>%
    filter(count > 1)

head(temp)

In [ ]:
df <- df %>%
    filter(!ssn %in% temp$ssn)

## Create a quarter variable

In [ ]:
df <- df %>%
    mutate(yr_month = ym(yr_month),
           Quarter = as.yearqtr(yr_month,"%Y%m%d"))
head(df)

In [ ]:
qry <- " use tr_tdc_2022;
"
DBI::dbExecute(con, qry)

DBI::dbWriteTable(
    conn = con,
    name = DBI::SQL("dbo.person_month_cleanA"), 
    value = df,
    overwrite  = TRUE
)

In [ ]:
rm(df)

## Create permanent table (in sql)

    select pm.*, cm.county,
        convert(date, concat(substring(month, 5, 6), '/', '01', '/', substring(month, 1, 4))) as yr_month,
        concat(substring(month, 1, 4), ' ', 'Q',  floor((convert(integer, substring(month, 5, 6)) + 2)/3)) as Quarter
    into tr_tdc_2022.dbo.person_month_clean
    from ds_in_fssa.dbo.person_month pm 
    WHERE ssn IN (SELECT DISTINCT SSN FROM tr_tdc_2022.dbo.person_month_cleanA)
    LEFT JOIN 
    ds_in_fssa.dbo.case_month cm 
    ON pm.caseid=cm.caseid AND cm.rptmn=pm.[MONTH]) 
    WHERE ssn IN (SELECT DISTINCT SSN FROM tr_tdc_2022.dbo.person_month_cleanA)

## Create 10% TANF table

In [ ]:
query <- "
with new_table as (
select distinct(ssn) 
    FROM tr_tdc_2022.dbo.person_month_clean a tablesample(10 percent) REPEATABLE (123)
    --group by ssn
)
select * from new_table
left join tr_tdc_2022.dbo.person_month_clean pm
on new_table.ssn = pm.ssn;
"

df2 <- dbGetQuery(con, query)[,2:20]   


In [ ]:
head(df2)

In [ ]:
qry <- " use tr_tdc_2022;
"
DBI::dbExecute(con, qry)

DBI::dbWriteTable(
    conn = con,
    name = DBI::SQL("dbo.person_month_clean_10pct"), 
    value = df2,
    overwrite  = TRUE
)

## Create 10% wage table

In [ ]:
query <- "
SELECT SSN, Empr_no, EIN, Year, Quarter, Wage
FROM ds_in_dwd.dbo.ui_wages
WHERE ssn IN (SELECT DISTINCT SSN FROM tr_tdc_2022.dbo.person_month_clean_10pct)
AND Wage > 0 
AND Wage IS NOT NULL;
"

wage <- dbGetQuery(con, query) 
head(wage)

In [ ]:
qry <- " use tr_tdc_2022;
"
DBI::dbExecute(con, qry)

DBI::dbWriteTable(
    conn = con,
    name = DBI::SQL("dbo.wage_clean_10pct"), 
    value = wage,
    overwrite  = TRUE
)